In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import csv
import pathlib
import bisect


In [ ]:
start_date = datetime.datetime(2023, 1, 1, 0, 0,0)
end_date = datetime.datetime.now()
#end_date = datetime.datetime(2020, 3, 20, 0, 0,0)

total_days = (end_date-start_date).days + 1
print(total_days)

total_price = np.zeros(total_days,dtype=float)

In [ ]:
def r2c(value):
    return float(int(value*100+0.5))/100

def scanFile(fileName) :
    ticker_name = os.path.basename(fileName).replace(".dat","")
    #print(ticker_name)

    arr = []
    f = open(fileName, 'rb')
    arr = pickle.load(f)
    f.close()

    cur_day = start_date
    
    start_index = bisect.bisect_left([r[0] for r in arr], cur_day.timestamp())

    for idx in range(start_index,len(arr)):
        
        row = arr[idx]
            
        date = datetime.datetime.fromtimestamp(row[0])
        o=float(row[1])
        h=float(row[2])
        l=float(row[3])
        c=float(row[4])
        
        if date > end_date:
            break
       
        if date > cur_day:
            cur_day = date
            
        index = (cur_day-start_date).days
        avg_price= (o+h+l+c)/4
        
        total_price[index]=total_price[index]+avg_price



In [ ]:
ticker2arr=[]

for p in pathlib.Path('bin').glob('**/*.dat') :
    scanFile(p)

In [ ]:
day_x=np.arange(0,len(total_price),dtype=float)

del_arr=[]
for i in range(0,len(total_price)):
    if total_price[i] == 0.0:
        del_arr.append(i)
        
day_x = np.delete(day_x, del_arr, axis=0)
total_price = np.delete(total_price, del_arr, axis=0)
    

In [ ]:
days=[]
for d in day_x:
    days.append((start_date + datetime.timedelta(days=d)).strftime('%Y-%m-%d'))
    
print(days[0])
print(days[-1])
print(len(total_price))
print(len(day_x))


In [ ]:
%matplotlib notebook
plt.plot(days,total_price)
#plt.xlim(20, 190)
#plt.ylim(0, 80000)
plt.show()
